Make the import

In [1]:
import numpy as np
import mne
import myonset as myo

# Import EMG data

Set EMG file name

In [2]:
fname  = './bva/example_data.vhdr'

Extract file name and create log file

Load raw data

In [3]:
raw = mne.io.read_raw_brainvision(fname,preload=True)

Extracting parameters from ./bva/example_data.vhdr...
Setting channel info structure...


C:\Users\Laure\AppData\Local\Temp\ipykernel_15324\1600125176.py:1: RuntimeWarning: No coordinate information found for channels ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'Status']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(fname,preload=True)


Reading 0 ... 118783  =      0.000 ...    58.000 secs...


C:\Users\Laure\AppData\Local\Temp\ipykernel_15324\1600125176.py:1: RuntimeWarning: Not setting positions of 5 misc channels found in montage:
['EXG1', 'EXG2', 'EXG3', 'EXG4', 'Status']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(fname,preload=True)


# Extract events 

In [4]:
trigger_events = mne.events_from_annotations(raw)[0]

Used Annotations descriptions: ['Comment/ActiveTwo MK2', 'Comment/CMS in range', 'Comment/Speed Mode 4', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 12', 'Stimulus/S 20', 'Stimulus/S 21', 'Stimulus/S 22', 'Stimulus/S 57', 'Stimulus/S100', 'Stimulus/S200', 'Stimulus/S254', 'Stimulus/S255']


Put mne events in our events structure

In [5]:
events = myo.Events(sample=trigger_events[:,0], code=trigger_events[:,2], chan=[-1]*trigger_events.shape[0],\
                    sf=raw.info['sfreq'])

# Pre-process signal

In [6]:
raw.info['ch_names']

['EXG1', 'EXG2', 'EXG3', 'EXG4', 'Status']

Set channel types to "eeg" (this required to apply bipolar reference)

In [7]:
channel_types = { ch: 'eeg' for ch in raw.info['ch_names']}
raw.set_channel_types(channel_types)

C:\Users\Laure\AppData\Local\Temp\ipykernel_15324\1868481596.py:2: RuntimeWarning: The unit for channel(s) EXG1, EXG2, EXG3, EXG4, Status has changed from NA to V.
  raw.set_channel_types(channel_types)


<RawBrainVision | example_data.dat, 5 x 118784 (58.0 s), ~4.5 MB, data loaded>

EMG bipolar reference 

In [8]:
myo.use_mne.bipolar_ref(raw, ['EXG1','EXG3'], ['EXG2','EXG4'], new_ch=['EMG_1','EMG_2'], copy=False)
raw.info['ch_names']

EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=2, n_times=118784
    Range : 0 ... 118783 =      0.000 ...    58.000 secs
Ready.
Added the following bipolar channels:
EMG_1, EMG_2


['Status', 'EMG_1', 'EMG_2']

Filter EMG

In [9]:
raw = myo.use_mne.apply_filter(raw, ch_names=['EMG_1','EMG_2'],low_cutoff = 10)

Get pre-processed raw data

In [10]:
data = myo.use_mne.get_data_array(raw, ch_names=['EMG_1','EMG_2'])
data.shape

(2, 118784)

# Save data and events

In [11]:
np.save('example_data.npy',data)

In [12]:
events.to_csv('example_events.csv')